# RFM ANALYSIS
## Overview
RFM is a classic Lifetime and Responsiveness segmentation model. It has been trialed and tested over the years and is a great starting point for any retailer including eCommerce companies looking to manage their customer base more proactively. 



### Goal: 

Those who exhibit common behaviors are in the same groups.sales and marketing specific to these groups. Feedback will be given on the development of techniques.


----

## Part One: Understanding the Background and Data

Dataset named Online Retail II UK based online store Between 01/12/2009 - 09/12/2011 includes sales. Souvenirs in the product catalog of this company items are included. As promotional items can also be considered.It is also known that most of its customers are wholesalers available.

https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

----


### The Data



#### online_retail_II.xlsx

Variable | Definition | Type of Variable
--- | ----------- | --------
InvoiceNo | Invoice number. 6-digit integral number uniquely assigned to each transaction.  | Nominal
          | **If this code starts with letter 'c', it indicates a cancellation.**
StockCode | Product (item) code. 5-digit integral number uniquely assigned to each distinct product.| Nominal
Description | Product (item) name. | Nominal
Quantity | The quantities of each product (item) per transaction. | Numeric
InvoiceDate | Invoice Date and time. The day and time when each transaction was generated.| Numeric
UnitPrice|  Unit price. Product price per unit in £ (in GBP) |  Numeric
CustomerID | Customer number. 5-digit integral number uniquely assigned to each customer.| Nominal
Country | Country name. The name of the country where each customer resides. | Nominal

**Import libraries**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Displaying all rows and columns
pd.set_option('display.max_columns', None); 
pd.set_option('display.max_rows', None);

# Configuring significant figures
pd.set_option('display.float_format', lambda x: '%.2f' % x)

**Read "online_retail.xlsx" file**

In [2]:
df_ = pd.read_excel("Data/online_retail_II.xlsx",
                    sheet_name="Year 2009-2010")

df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


**Counting number of missing/null values in every variable**

In [3]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

**Total number of unique products**

In [4]:
df["Description"].nunique()

4681

**Total quantities of unique products**

In [5]:
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

**The most ordered product**

In [6]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


**Total Invoice quantity**

In [7]:
df["Invoice"].nunique()

28816

**Income per invoice**

**Note = Multiple invoices present in some rows**

**Removing cancelled transcations (which are our outliers in the dataset)**

In [8]:
df = df[~df["Invoice"].str.contains("C", na=False)]

**Creating Total Price variable from multiplication of quantities and prices**

In [9]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

**Which products are the most expensive?**

In [10]:
df.sort_values("Price", ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09,NaN,United Kingdom,25111.09
517955,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,13541.33
135013,502263,M,Manual,1,2010-03-23 15:22:00,10953.50,12918.00,United Kingdom,10953.50
135015,502265,M,Manual,1,2010-03-23 15:28:00,10953.50,NaN,United Kingdom,10953.50
342147,522796,M,Manual,1,2010-09-16 15:12:00,10468.80,NaN,United Kingdom,10468.80


**Order per country**

In [11]:
df["Country"].value_counts()

United Kingdom          477058
EIRE                      9460
Germany                   7661
France                    5532
Netherlands               2730
Spain                     1235
Switzerland               1170
Portugal                  1061
Belgium                   1038
Sweden                     887
Channel Islands            821
Italy                      710
Australia                  630
Cyprus                     541
Austria                    524
Greece                     512
Denmark                    418
United Arab Emirates       399
Norway                     365
Finland                    347
Unspecified                306
USA                        230
Poland                     182
Malta                      170
Japan                      164
Lithuania                  154
Singapore                  117
RSA                        110
Bahrain                    106
Canada                      77
Thailand                    76
Hong Kong                   74
Israel  

**Income per country**

In [12]:
df.groupby("Country").agg({"TotalPrice": "sum"}).sort_values("TotalPrice", ascending=False).head()

,TotalPrice
Country,
United Kingdom,8709203.67
EIRE,380977.82
Netherlands,268786.00
Germany,202395.32
France,147211.49


**Data Preperation**

In [13]:
df.isnull().sum()
df.dropna(inplace=True)
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,407695.00,13.59,96.84,1.00,1.00,1.00,1.00,2.00,5.00,12.00,24.00,36.00,144.00,19152.00
Price,407695.00,3.29,34.76,0.00,0.29,0.42,0.65,1.25,1.95,3.75,6.75,8.50,14.95,10953.50
Customer ID,407695.00,15368.50,1679.80,12346.00,12435.00,12731.00,13044.00,13997.00,15321.00,16812.00,17706.00,17913.00,18196.00,18287.00
TotalPrice,407695.00,21.66,77.15,0.00,0.65,1.25,2.10,4.95,11.90,19.50,35.70,67.50,201.60,15818.40


## Part Two: Calculating RFM Metrics

Recency, Frequency, Monetary ifadelerinin baş harflerinden oluşur.

Müşterilerin satın alma alışkanlıkları üzerinden pazarlama ve satış stratejileri belirlemeye yardımcı olan bir tekniktir.

Recency (yenilik): Müşterinin son satın almasından bugüne kadar geçen süre

Frequency (Sıklık): Toplam satın alma sayısı.

Monetary (Parasal Değer): Müşterinin yaptığı toplam harcama.


In [14]:
df["InvoiceDate"].max()

today_date = dt.datetime(2010, 12, 11)

rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: len(num),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})


rfm.columns = ['Recency', 'Frequency', 'Monetary']

rfm = rfm[(rfm["Monetary"]) > 0 & (rfm["Frequency"] > 0)]

**Calculating RFM Scores**

In [15]:
# Recency
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5])

rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])


rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))


rfm[rfm["RFM_SCORE"] == "555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12415.00,11,212,19543.84,5,5,5,555
12431.00,9,170,4370.52,5,5,5,555
12433.00,2,286,7205.39,5,5,5,555
12471.00,10,678,20139.74,5,5,5,555
12472.00,5,572,11308.48,5,5,5,555


**Naming & Analysing RFM Segments**

In [16]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

rfm

rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)

rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
df[["Customer ID"]].nunique()
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])

rfm[rfm["Segment"] == "Need_Attention"].head()
rfm[rfm["Segment"] == "Need_Attention"].index

new_df = pd.DataFrame()

new_df["Need_Attention"] = rfm[rfm["Segment"] == "Need_Attention"].index

new_df.to_csv("ExportedCustomers/Need_Attention.csv")

In [17]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00,165,33,372.86,2,3,2,232,At_Risk
12347.00,3,71,1323.32,5,4,4,544,Champions
12348.00,74,20,222.16,2,2,1,221,Hibernating
12349.00,43,102,2671.14,3,4,5,345,Loyal_Customers
12351.00,11,21,300.93,5,2,2,522,Potential_Loyalists


In [18]:
rfm[rfm["Segment"] == "Hibernating"].describe().T
#Hibernating has a great M value which should be revived.
rfm[rfm["Segment"] == "Need_Attention"].describe().T
#These are worse than hibernating.
rfm[rfm["Segment"] == "Promising"].describe().T
#Promising should be more "promising" Their recency value is too low but frequency is okayish

#Can we able to save this sinking ship from its fate?

,count,mean,std,min,25%,50%,75%,max
Recency,112.00,24.99,5.93,16.00,21.00,25.00,30.00,36.00
Frequency,112.00,8.62,4.15,1.00,5.75,9.00,12.00,15.00
Monetary,112.00,456.51,749.37,24.35,132.84,224.25,412.41,4742.00


**Exporting the loyal customers**

In [19]:
exported_df = pd.DataFrame()

exported_df["Loyal_Customers"] = rfm[rfm["Segment"] == "Loyal_Customers"].index

exported_df.to_csv("ExportedCustomers/Loyal_Customers.csv")
rfm.to_csv("ExportedCustomers/rfm.csv")

group = rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(
    ["mean", "count", "median"])


group.to_csv("ExportedCustomers/groupp_agg.csv")